In [1]:
import requests

In [2]:
import bs4

In [ ]:
def get_ids(number):
  url = f'https://hi-tech.mail.ru/mobile-catalog/?page={number}'
  req = requests.get(url)
  soup = bs4.BeautifulSoup(req.text, 'html')
  phones = soup.findAll('div', attrs={'class': 'p-catalog-card__title'})
  answer = []
  for i in phones:
    answer.append(i.find('a')['href'][-17:-9])
  
  return answer

In [15]:
import time

In [ ]:
list_of_ids = []

In [ ]:
for i in range(1, 83):
  list_of_ids.extend(get_ids(i))
  time.sleep(1)

In [7]:
correct_ids = []
for i in list_of_ids:
  if all(j.isnumeric() for j in i):
    correct_ids.append(i)

In [8]:
len(correct_ids)

3846

In [ ]:
with open('ids.txt', 'w') as file:
  file.write(' '.join(correct_ids))

In [5]:
with open('ids.txt', 'r') as file:
  list_of_ids = file.read().split(' ')

Проверяем если страница пустая:

In [ ]:
len(str(requests.get('https://hi-tech.mail.ru/ajax/1105154-catalog/feedback/list/?page=31').json())) < 38

False

In [ ]:
len(str(requests.get('https://hi-tech.mail.ru/ajax/1105154-catalog/feedback/list/?page=32').json())) < 38

True

In [10]:
import datetime

In [31]:
def get_data(id_number):

  item_set = set()
  n = 38
  page = 0
  while True:
    url = f'https://hi-tech.mail.ru/ajax/{int(id_number)}-catalog/feedback/list/?page={page}'
    data = requests.get(url).json()
    n = len(str(data))
    if n < 38:
      break
    else:
      data = data['data']
      for i in data:
        text = ' '.join(i['text'][j]['text'] for j in range(len(i['text'])))
        text = text.replace('/', '').replace('<p>', '').replace('\r\n', ' ')
        star = i['star']
        item_set.add((text, star))
        
      page += 1
    
    time.sleep(1)
  
  return item_set
  

In [46]:
rated_reviews = set()

In [47]:
now = datetime.datetime.now()
for i in list_of_ids:
    if len(rated_reviews) >= 6000:
      break
    else:
      item_review = get_data(i)
      rated_reviews.update(item_review)
      print(f'{i}: {len(rated_reviews)}. {datetime.datetime.now() - now}')
      time.sleep(1)

print(f'Operation executed. Time consumed: {datetime.datetime.now() - now}')

14718700: 7. 0:00:05.169467
14718702: 14. 0:00:11.334649
14673056: 18. 0:00:15.972464
14704129: 18. 0:00:17.547900
14643007: 89. 0:00:43.805069
14668690: 112. 0:00:54.595379
14580352: 223. 0:01:33.691558
14586234: 245. 0:01:44.657528
14473437: 342. 0:02:18.774896
14767776: 342. 0:02:20.364663
14767774: 342. 0:02:21.952114
14646538: 392. 0:02:42.362779
14527722: 578. 0:03:45.006272
14603587: 740. 0:04:39.810583
14486768: 783. 0:04:57.215794
14668716: 830. 0:05:15.943770
14658180: 846. 0:05:25.263475
14553539: 989. 0:06:13.563539
14668755: 1013. 0:06:24.386601
14515949: 1084. 0:06:50.890190
14486766: 1158. 0:07:17.422097
14554018: 1178. 0:07:26.813794
14404837: 1224. 0:07:45.645784
14655743: 1244. 0:07:54.934155
14718700: 1244. 0:08:01.116556
14718702: 1244. 0:08:07.302533
14673056: 1244. 0:08:11.942085
14704129: 1244. 0:08:13.530829
14643007: 1244. 0:08:40.008334
14668690: 1244. 0:08:50.929828
14580352: 1244. 0:09:29.815547
14586234: 1244. 0:09:40.642561
14473437: 1244. 0:10:14.851144
1

In [48]:
import pandas as pd

In [51]:
texts, stars = [], []

In [52]:
for i in rated_reviews:
  texts.append(i[0])
  stars.append(i[1])

In [53]:
texts[0]

'Красивый, отличный экран, богатый функционал Как бы не оптимизировали, лаги при использовании есть практически постоянно. Плавности iOs тут нет и не было. В целом устройством доволен. После 10 лет использования устройств на iOS рад новым впечатлениям. '

In [54]:
stars[0]

5

In [57]:
pd.DataFrame({'text': texts, 'rate': stars}, index=None).to_csv('rated_reviews.csv')